In [2]:
import csv
import urllib
import arcgis
import geopandas as gpd
from arcgis.gis import GIS
import numpy as np
from shapely.geometry import Point
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.features.find_locations import find_existing_locations
from arcgis.features import SpatialDataFrame
from arcgis.features import use_proximity
from arcgis.geoenrichment import *
import pandas as pd
from arcgis.features.manage_data import merge_layers
from arcgis.geoenrichment import standard_geography_query, enrich
import seaborn as sns;
sns.set(color_codes=True)
import math
import datetime
import matplotlib.pyplot as plt
from arcgis.features import summarize_data
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [3]:
# Yicen Ma
gis = GIS("https://ucsdonline.maps.arcgis.com/home/item.html?id=01cee8d17yim095@ucsd.edue2446739d57e7714c693624", client_id='yYvN0h2gOFqtCZRA')
print("Successfully logged in as: " + gis.properties.user.username)

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://ucsdonline.maps.arcgis.com/sharing/rest//oauth2/authorize?response_type=code&client_id=yYvN0h2gOFqtCZRA&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=yi2LfLOTzf1IOUJXjtAOXR2AWvdwWS
Enter code obtained on signing in using SAML: ········


/opt/conda/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Successfully logged in as: yim095_UCSDOnline


In [4]:
#zipcode with comfired cases
comfired_zip = pd.read_csv('confired cases in zipcode.csv')
comfired_zip.head()

,updated,date through,zipcode,confirmed_cases\
0,4/5/20,3/30/20,91902,5\
1,4/5/20,3/30/20,91909,1\
2,4/5/20,3/30/20,91910,11\
3,4/5/20,3/30/20,91911,11\
4,4/5/20,3/30/20,91913,10\


In [5]:
#clean the csv
comfired_zip['confirmed_cases\\'] = comfired_zip['confirmed_cases\\'].apply(lambda x: x.strip('\\'))
comfired_zip['confirmed_cases'] = comfired_zip['confirmed_cases\\']
comfired_zip = comfired_zip.drop('confirmed_cases\\', axis = 1)
list(comfired_zip[comfired_zip['zipcode']=='Unknown*'].index)

[81, 168, 259, 356, 451, 546, 641, 730, 819, 909, 999, 1090]

In [6]:
# drop the unknow and total
index1 = list(comfired_zip[comfired_zip['zipcode']=='Unknown*'].index)
index1 = index1+list(comfired_zip[comfired_zip['zipcode']=='Unknown'].index)
index1 = index1 + list(comfired_zip[comfired_zip['zipcode']=='Total'].index)
index1 = index1 + list(comfired_zip[comfired_zip['zipcode']=='TOTAL'].index)

In [7]:
comfired_zip_new = comfired_zip.drop(index1).reset_index(drop=True)
sd_zipcode = gis.content.get("8ab0aaa00ad14e6ebb3a27b9e37aa02c")
sd_zipcode_sdf = pd.DataFrame.spatial.from_layer(sd_zipcode.layers[0])
comfired_zip_new.head()

,updated,date through,zipcode,confirmed_cases
0,4/5/20,3/30/20,91902,5
1,4/5/20,3/30/20,91909,1
2,4/5/20,3/30/20,91910,11
3,4/5/20,3/30/20,91911,11
4,4/5/20,3/30/20,91913,10


In [8]:
dict_zip = pd.Series(sd_zipcode_sdf.SHAPE.values,index=sd_zipcode_sdf.ZIP).to_dict()

In [9]:
notin = []
for i in list(comfired_zip_new['zipcode'].unique()):
    i = int(i)
    if i not in list(dict_zip.keys()):
        notin.append(i)

In [10]:
notin.sort()
notin

[91909, 92039, 92085, 92088, 92092, 92147, 92159, 92168, 92196]

In [11]:
for k in notin:
    if k == 91909:
        x = comfired_zip_new[comfired_zip_new['zipcode'] == '91909'].index
        old = comfired_zip_new.iloc[x]
        new = comfired_zip_new.iloc[comfired_zip_new[comfired_zip_new['zipcode'] == '91911'].index]
        for i in list(old.index):
            date = old.loc[i][1]
            value = new.loc[new['date through'] == date, 'confirmed_cases'].values[0]
            value1 = old.loc[old['date through'] == date, 'confirmed_cases'].values[0]
            new.loc[new['date through'] == date, 'confirmed_cases'] = str(int(value)+int(value1))
        comfired_zip_new.loc[new.index,'confirmed_cases'] = new['confirmed_cases']
    if k == 92039:
        x = comfired_zip_new[comfired_zip_new['zipcode'] == '92039'].index
        old = comfired_zip_new.iloc[x]
        new = comfired_zip_new.iloc[comfired_zip_new[comfired_zip_new['zipcode'] == '92037'].index]
        for i in list(old.index):
            date = old.loc[i][1]
            value = new.loc[new['date through'] == date, 'confirmed_cases'].values[0]
            value1 = old.loc[old['date through'] == date, 'confirmed_cases'].values[0]
            new.loc[new['date through'] == date, 'confirmed_cases'] = str(int(value)+int(value1))
        comfired_zip_new.loc[new.index,'confirmed_cases'] = new['confirmed_cases']
    if k == 92196:
        x = comfired_zip_new[comfired_zip_new['zipcode'] == '92196'].index
        old = comfired_zip_new.iloc[x]
        new = comfired_zip_new.iloc[comfired_zip_new[comfired_zip_new['zipcode'] == '92126'].index]
        for i in list(old.index):
            date = old.loc[i][1]
            value = new.loc[new['date through'] == date, 'confirmed_cases'].values[0]
            value1 = old.loc[old['date through'] == date, 'confirmed_cases'].values[0]
            new.loc[new['date through'] == date, 'confirmed_cases'] = str(int(value)+int(value1))
        comfired_zip_new.loc[new.index,'confirmed_cases'] = new['confirmed_cases']
    if k == 92092 or k == 92147:
        pass
    if k ==92168:
        x = comfired_zip_new[comfired_zip_new['zipcode'] == '92168'].index
        old = comfired_zip_new.iloc[x]
        new = comfired_zip_new.iloc[comfired_zip_new[comfired_zip_new['zipcode'] == '92111'].index]
        for i in list(old.index):
            date = old.loc[i][1]
            value = new.loc[new['date through'] == date, 'confirmed_cases'].values[0]
            value1 = old.loc[old['date through'] == date, 'confirmed_cases'].values[0]
            new.loc[new['date through'] == date, 'confirmed_cases'] = str(int(value)+int(value1))
        comfired_zip_new.loc[new.index,'confirmed_cases'] = new['confirmed_cases']
    if k ==92088:
        x = comfired_zip_new[comfired_zip_new['zipcode'] == '92088'].index
        old = comfired_zip_new.iloc[x]
        new = comfired_zip_new.iloc[comfired_zip_new[comfired_zip_new['zipcode'] == '92028'].index]
        for i in list(old.index):
            date = old.loc[i][1]
            value = new.loc[new['date through'] == date, 'confirmed_cases'].values[0]
            value1 = old.loc[old['date through'] == date, 'confirmed_cases'].values[0]
            new.loc[new['date through'] == date, 'confirmed_cases'] = str(int(value)+int(value1))
        comfired_zip_new.loc[new.index,'confirmed_cases'] = new['confirmed_cases']
    if k ==92085:
        x = comfired_zip_new[comfired_zip_new['zipcode'] == '92085'].index
        old = comfired_zip_new.iloc[x]
        new = comfired_zip_new.iloc[comfired_zip_new[comfired_zip_new['zipcode'] == '92084'].index]
        for i in list(old.index):
            date = old.loc[i][1]
            value = new.loc[new['date through'] == date, 'confirmed_cases'].values[0]
            value1 = old.loc[old['date through'] == date, 'confirmed_cases'].values[0]
            new.loc[new['date through'] == date, 'confirmed_cases'] = str(int(value)+int(value1))
        comfired_zip_new.loc[new.index,'confirmed_cases'] = new['confirmed_cases']
    if k ==92159:
        x = comfired_zip_new[comfired_zip_new['zipcode'] == '92159'].index
        old = comfired_zip_new.iloc[x]
        new = comfired_zip_new.iloc[comfired_zip_new[comfired_zip_new['zipcode'] == '92119'].index]
        for i in list(old.index):
            date = old.loc[i][1]
            value = new.loc[new['date through'] == date, 'confirmed_cases'].values[0]
            value1 = old.loc[old['date through'] == date, 'confirmed_cases'].values[0]
            new.loc[new['date through'] == date, 'confirmed_cases'] = str(int(value)+int(value1))
        comfired_zip_new.loc[new.index,'confirmed_cases'] = new['confirmed_cases']
    

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/li

In [12]:
index2 = []
for i in  notin:
    index2 = index2 + list(comfired_zip_new[comfired_zip_new['zipcode'] ==str(i)].index)

In [13]:
comfired_zip_new = comfired_zip_new.drop(index2).reset_index(drop=True)

In [14]:
comfired_zip_new['SHAPE'] = comfired_zip_new['zipcode'].apply(lambda x: dict_zip[int(x)])
comfired_zip_new.head()

,updated,date through,zipcode,confirmed_cases,SHAPE
0,4/5/20,3/30/20,91902,5,"{'rings': [[[-13021180.7298882, 3856659.042188..."
1,4/5/20,3/30/20,91910,11,"{'rings': [[[-13031455.2456078, 3848954.227753..."
2,4/5/20,3/30/20,91911,12,"{'rings': [[[-13029541.3960757, 3846081.710701..."
3,4/5/20,3/30/20,91913,10,"{'rings': [[[-13021187.1936931, 3848402.699601..."
4,4/5/20,3/30/20,91914,2,"{'rings': [[[-13018754.0621962, 3853132.189266..."


In [15]:
# comfired_case_in_zipcode = comfired_zip_new.spatial.to_featurelayer(title='comfired_zip_new',gis=gis,tags='dsc198')
# group=gis.groups.search('title:DSC198 Covid-19')
# comfired_case_in_zipcode.share(groups=group)

In [16]:
comfired_zip_new = comfired_zip_new.drop('updated',axis = 1)
comfired_zip_new = comfired_zip_new.drop_duplicates()

In [18]:
# change to dataframe
code = comfired_zip_new[comfired_zip_new['zipcode']=='91902']
confired_with_date = pd.DataFrame(columns = code['date through'].unique())
confired_with_date

,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,...,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20


In [19]:
for i in list(comfired_zip_new.index):
    value = comfired_zip_new.loc[i]
    confired_with_date.loc[value[1],value[0]]=int(value[2])

In [20]:
modified_confired_with_date = confired_with_date.fillna(0)
modified_confired_with_date['ZIP'] = modified_confired_with_date.index
modified_confired_with_date = modified_confired_with_date.reset_index(drop = True)
sd_zipcode = sd_zipcode_sdf[['ZIP','SHAPE']]
sd_zipcode['ZIP'] = sd_zipcode_sdf['ZIP'].apply(str)
modified_confired_with_date = modified_confired_with_date.merge(sd_zipcode, on='ZIP',how = 'inner')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [21]:
modified_confired_with_date.head()

,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,...,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,ZIP,SHAPE
0,5,8,8,9,10,11,14,16,16,16,...,16,17,17,18,18,18,18,22,91902,"{'rings': [[[-13021180.7298882, 3856659.042188..."
1,11,17,21,23,28,30,34,39,43,44,...,57,61,66,66,68,68,70,73,91910,"{'rings': [[[-13031455.2456078, 3848954.227753..."
2,12,14,17,22,25,30,36,38,47,52,...,72,74,80,83,88,94,98,106,91911,"{'rings': [[[-13029541.3960757, 3846081.710701..."
3,10,14,19,20,21,25,26,28,32,34,...,37,37,39,40,41,43,43,47,91913,"{'rings': [[[-13021187.1936931, 3848402.699601..."
4,2,2,2,2,3,4,6,7,8,8,...,9,11,14,14,14,14,14,14,91914,"{'rings': [[[-13018754.0621962, 3853132.189266..."


In [22]:
# comfired_case_in_zipcode =modified_confired_with_date.spatial.to_featurelayer(title='modified_confired_with_date',gis=gis,tags='dsc198')
# group=gis.groups.search('title:DSC198 Covid-19')
# comfired_case_in_zipcode.share(groups=group)